In [ ]:
import socketio
import requests
from os import environ as env

In [ ]:
sio = socketio.Client()

In [ ]:
resolution = None

@sio.on('resolution', namespace='/control')
def handle_resolution(w, h):
    print('resolution', w, h)
    resolution = (w, h)

In [ ]:
hostname = env['HOST'].strip()
sio.connect('http://%s:5000' % hostname, namespaces=['/control'])

In [ ]:
def on_response(*resp):
    print('response:', *resp)
    
sio.emit('handshake', namespace='/control', callback=on_response)

In [ ]:
# sio.emit('highlight', data=(5, 6), namespace='/control', callback=on_response)

In [ ]:
import requests as rq
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def read_image_from_url(url):
    r = rq.get(url)
    assert r.ok, 'Failed to fetch the image'
    img_str = r.content

    img_arr = np.frombuffer(img_str, np.uint8)

    img = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
    assert img is not None, 'Failed to read the image'
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img);
    
    return img

img = read_image_from_url('https://t4.rbxcdn.com/05b0ee639de3e3e9b40dcbbad11a78b8')

In [ ]:
img_out = cv2.resize(img, (30,30))
plt.imshow(img_out);

In [ ]:
def display_image(img):
    image_payload = {
        'image': img.tobytes(),
        'width': img.shape[0],
        'height': img.shape[1]
    }
    sio.emit('show_image', data=image_payload, namespace='/control', callback=on_response)

In [ ]:
display_image(img_out)